## clinical variable retrieval codebase.
each querey is referencing a sql querey on my computer for ALL patients in the database, then generating a dataframe, then paring that dataframe down to only the patients/icustay_id in our cohort. it's inefficient but works. 

In [9]:
import requests
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import psycopg2
import collections
import asyncio
import getpass
import re
from datetime import datetime as dt
import os,sys,re
import urllib3
import prettytable
from collections import Counter
import seaborn as sns
import random

from sklearn.externals.joblib import Memory
memory = Memory(cachedir='/tmp', verbose=0)
#@memory.cache above any def fxn.

%matplotlib inline
plt.style.use('ggplot')

from notebook.services.config import ConfigManager
cm = ConfigManager()
cm.update('livereveal', {
        'width': 1024,
        'height': 768,
        'scroll': True,
})

%load_ext autotime

In [2]:
import pandas as pd 

In [79]:
#patients of interest from rotation_cohort_generation


final_pt_df2 = pd.read_csv('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/final_pt_df2_5-04-18.csv', index_col=0)
patients= list(final_pt_df2['subject_id'].unique())
icustay_id= list(final_pt_df2['icustay_id'].unique())

time: 56.8 ms


### todo 5/8/2018:
    would be nice to ping server in each querey for only patients or icustay_id in the final cohort,
    would greatly reduce querey time.

## Fetch all responses within one Client session first

In [13]:
#I need help understanding what this is doing, can you elaborate?
#what are the fetch and run functions? This is used in pinging a api, right?

time: 626 µs


In [5]:
import asyncio
from aiohttp import ClientSession
import time
import json

async def fetch(url, session):
    async with session.get(url) as response:
        return await response.read()

async def run(url_lst):
    tasks = []

    # Fetch all responses within one Client session,
    # keep connection alive for all requests.
    start = time.perf_counter()
    
    async with ClientSession() as session:
        for url in url_lst:
            task = asyncio.ensure_future(fetch(url, session))
            tasks.append(task)

        responses = await asyncio.gather(*tasks)
        # you now have all response bodies in this variable
#         print(responses)

    duration = time.perf_counter() - start
    print('total time: {}'.format(duration))
    return responses

## Access MIMIC database and convert it to dataframe in Pandas

In [157]:
import psycopg2
# conn = psycopg2.connect(dbname="mimic", user="student", password="lipup", host="luolabwkstn01.fsm.northwestern.edu", port="5432")
# cur=conn.cursor()

conn = psycopg2.connect(dbname="mimic", user="student", host="luolabwkstn01.fsm.northwestern.edu", port="5432",
                        password=getpass.getpass(prompt='Password:'.format()))
cur=conn.cursor()

query_schema = 'SET search_path to ' + "mimiciii" + ';'

Password:········
time: 2.08 s


In [14]:
def sql_exe_show(sql_sentence):
    cur.execute(sql_sentence)
    rows = cur.fetchall()
    col = []
    for i in range(len(cur.description)):
        col.append(cur.description[i][0])
    table = pd.DataFrame(rows,columns=col)
    return table

time: 158 ms


In [80]:
#input the sql_exe_show object and get dataframe for only patients in patient list out. 
def sql_to_df_icu(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['icustay_id'].isin(icustay_id)]
    return sql_exe_show_df

def sql_to_df_patients(sql_exe_show_obj):
    sql_exe_show_df= pd.DataFrame(data=sql_exe_show_obj)
    sql_exe_show_df=sql_exe_show_df[sql_exe_show_df['subject_id'].isin(patients)]
    return sql_exe_show_df

time: 7.96 ms


## Results in .csv

In [ ]:
#where are these tables? i don't see any code generating this? 

In [95]:
index = [] 
index.append(echoTable.columns)
index.append(heightFirstDayTable.columns)
index.append(weightFirstDayTable.columns)
index.append(vitalsFirstDayTable.columns)
index.append(labsFirstDayTable.columns)
index.append(bgFirstDayTable.columns)
index.append(gcsFirstDayTable.columns)
index.append(rrtFirstDayTable.columns)
index.append(urineFirstDayTable.columns)
pd.DataFrame(index).to_csv('index.csv')


NameError: name 'vitalsFirstDay' is not defined

time: 46.7 ms


In [99]:
# index = [] 
# index.append(echoDf.columns)
# index.append(heightFirstDayDf.columns)
# index.append(weightFirstDayDf.columns)
# index.append(vitalsFirstDayDf.columns)
# index.append(labsFirstDayDf.columns)
# index.append(bgFirstDayDf.columns)
# index.append(gcsFirstDayDf.columns)
# index.append(rrtFirstDayDf.columns)
# index.append(urineFirstDayDf.columns)
pd.DataFrame(echoDf).to_csv('clinical_csv/echoDf.csv')
pd.DataFrame(heightFirstDayDf).to_csv('clinical_csv/heightFirstDayDf.csv')
pd.DataFrame(weightFirstDayDf).to_csv('clinical_csv/weightFirstDayDf.csv')
pd.DataFrame(vitalsFirstDayDf).to_csv('clinical_csv/vitalsFirstDayDf.csv')
pd.DataFrame(labsFirstDayDf).to_csv('clinical_csv/labsFirstDayDf.csv')
pd.DataFrame(bgFirstDayDf).to_csv('clinical_csv/bgFirstDayDf.csv')
pd.DataFrame(gcsFirstDayDf).to_csv('clinical_csv/gcsFirstDayDf.csv')
pd.DataFrame(rrtFirstDayDf).to_csv('clinical_csv/rrtFirstDayDf.csv')
pd.DataFrame(urineFirstDayDf).to_csv('clinical_csv/urineFirstDayDf.csv')


time: 2.75 s


## Echodata in Noteevents

In [98]:
f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/echodata.sql', 'r') 
echodata = f.read()
cur.execute('rollback')
cur.execute(echodata)
echoTable = sql_exe_show('select * from echodata;')
echoDf= sql_to_df_patients(echoTable)
echoDf[0:500]

,row_id,subject_id,hadm_id,chartdate,charttime,indication,height,weight,bsa,bp,bpsys,bpdias,hr,status,test,doppler,contrast,technicalquality
0,59653,31038,174978.0,2102-06-14,2102-06-14 13:30:00,Endocarditis.,74,165,2.00,102/72,102,72,86,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
2,59655,54190,188571.0,2158-02-16,2158-02-16 13:48:00,Cerebrovascular event/TIA. Streptococcal bacte...,72,187,2.07,147/87,147,87,90,Inpatient,Portable TEE (Complete),Full Doppler and color Doppler,None,Adequate
3,59656,5771,185291.0,2173-09-02,2173-09-02 09:47:00,78 year-old man with atrial fibrillation off a...,66,130,1.67,163/63,163,63,80,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
6,59659,82208,188268.0,2200-10-22,2200-10-22 15:14:00,"Acute MR worsening,ESRD, on HD, NSLS lung Ca, ...",59,136,1.57,126/61,126,61,95,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
7,59660,82208,NaN,2200-01-02,2200-01-02 11:23:00,Congestive heart failure. Hypertension. Shortn...,59,143,1.60,170/82,170,82,92,Inpatient,TTE (Complete),Full Doppler and color Doppler,None,Adequate
8,59669,15472,NaN,2176-05-28,2176-05-28 08:58:00,Left ventricular function. Shortness of breath...,72,200,2.13,122/90,122,90,None,Inpatient,TTE(Focused views),Focused pulse and color flow,None,Suboptimal
9,59670,15472,118185.0,2175-09-19,2175-09-19 09:58:00,Left ventricular function.,None,None,None,123/76,123,76,None,Inpatient,Portable TEE(Complete),Complete pulse and color flow,None,Adequate
11,59672,7429,110364.0,2126-09-09,2126-09-09 16:06:00,Endocarditis.,64,150,1.73,122/50,122,50,90,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
15,59770,96774,NaN,2174-03-31,2174-03-31 16:30:00,Dyspnea. Rule out pulmonary hypertension.,65,216,2.05,122/84,122,84,93,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Adequate
16,59771,96774,129414.0,2174-02-14,2174-02-14 09:45:00,Shortness of breath. Left ventricular functio...,64,206,1.98,100/66,100,66,102,Inpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal


time: 1min 3s


In [52]:
echoTable[0:500]

,row_id,subject_id,hadm_id,chartdate,charttime,indication,height,weight,bsa,bp,bpsys,bpdias,hr,status,test,doppler,contrast,technicalquality
0,59653,31038,174978.0,2102-06-14,2102-06-14 13:30:00,Endocarditis.,74,165,2.00,102/72,102,72,86,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
1,59654,70150,156140.0,2191-05-10,2191-05-10 15:08:00,"CHF, A-FIB",66,263,2.25,107/81,107,81,95,Outpatient,TTE (Complete),Full Doppler and color Doppler,None,Suboptimal
2,59655,54190,188571.0,2158-02-16,2158-02-16 13:48:00,Cerebrovascular event/TIA. Streptococcal bacte...,72,187,2.07,147/87,147,87,90,Inpatient,Portable TEE (Complete),Full Doppler and color Doppler,None,Adequate
3,59656,5771,185291.0,2173-09-02,2173-09-02 09:47:00,78 year-old man with atrial fibrillation off a...,66,130,1.67,163/63,163,63,80,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
4,59657,80030,100442.0,2119-06-07,2119-06-07 14:59:00,Left ventricular function.,63,126,1.59,88/53,88,53,115,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
5,59658,65696,167705.0,2196-09-16,2196-09-16 01:19:00,Left ventricular function. Myocardial infarction.,73,200,2.15,100/42,100,42,89,Inpatient,Portable TTE (Focused views),Limited Doppler and color Doppler,None,Adequate
6,59659,82208,188268.0,2200-10-22,2200-10-22 15:14:00,"Acute MR worsening,ESRD, on HD, NSLS lung Ca, ...",59,136,1.57,126/61,126,61,95,Inpatient,Portable TTE (Complete),Full Doppler and color Doppler,None,Adequate
7,59660,82208,NaN,2200-01-02,2200-01-02 11:23:00,Congestive heart failure. Hypertension. Shortn...,59,143,1.60,170/82,170,82,92,Inpatient,TTE (Complete),Full Doppler and color Doppler,None,Adequate
8,59669,15472,NaN,2176-05-28,2176-05-28 08:58:00,Left ventricular function. Shortness of breath...,72,200,2.13,122/90,122,90,None,Inpatient,TTE(Focused views),Focused pulse and color flow,None,Suboptimal
9,59670,15472,118185.0,2175-09-19,2175-09-19 09:58:00,Left ventricular function.,None,None,None,123/76,123,76,None,Inpatient,Portable TEE(Complete),Complete pulse and color flow,None,Adequate


time: 68.6 ms


## Weight

In [85]:
# -- This query extracts weights for adult ICU patients on their first ICU day.
# -- It does *not* use any information after the first ICU day, as weight is
# -- sometimes used to monitor fluid balance.

f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/weight.sql', 'r') 
weightSQL = f.read()
cur.execute('rollback')
cur.execute(weightSQL)
weightFirstDayTable = sql_exe_show('select * from weightfirstday;')
weightFirstDayDf= sql_to_df_icu(weightFirstDayTable)
weightFirstDayDf[0:500]


,icustay_id,weight,weight_admit,weight_daily,weight_echoinhosp,weight_echoprehosp
7,200012,51.20,51.200001,51.200001,NaN,NaN
10,200019,65.00,65.000000,NaN,NaN,NaN
21,200036,79.00,NaN,79.000000,NaN,NaN
33,200055,56.00,56.000000,NaN,NaN,NaN
34,200059,54.00,54.000000,60.000000,NaN,NaN
37,200063,None,NaN,NaN,NaN,NaN
38,200065,120.00,120.000000,NaN,108.862080,NaN
51,200087,95.10,95.100000,NaN,NaN,NaN
54,200095,None,NaN,NaN,NaN,NaN
56,200099,66.00,66.000000,76.700000,NaN,NaN


time: 2.61 s


## Height

In [86]:
# -- This query extracts heights for adult ICU patients.
# -- It uses all information from the patient's first ICU day.
# -- This is done for consistency with other queries - it's not necessarily needed.
# -- Height is unlikely to change throughout a patient's stay.

f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/height.sql', 'r')
heightSQL = f.read()
cur.execute('rollback') not sure what this does
cur.execute(heightSQL)
heightFirstDayTable = sql_exe_show('select * from heightfirstday;')
heightFirstDayDf= sql_to_df_icu(heightFirstDayTable)
heightFirstDayDf[0:500]




,icustay_id,height,height_chart,height_echo
4,216859,172.72,NaN,172.72
5,203487,185.42,185.42,182.88
7,254478,182.88,182.88,165.10
8,296681,182.88,NaN,182.88
19,252348,173.99,173.99,170.18
21,216609,172.72,172.72,172.72
22,272667,172.72,172.72,172.72
30,252051,175.26,175.26,182.88
54,231823,152.40,NaN,152.40
73,249202,170.18,NaN,170.18


time: 1.25 s


# Labs

In [87]:
# -- This query pivots lab values taken in the first 24 hours of a patient's stay
# -- Have already confirmed that the unit of measurement is always the same: null or the correct unit


f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/labs.sql', 'r')
labsSQL = f.read()
cur.execute('rollback')
cur.execute(labsSQL)
labsFirstDayTable = sql_exe_show('select * from labsfirstday;')
labsFirstDayDf= sql_to_df_icu(labsFirstDayTable)
labsFirstDayDf[0:500]



,subject_id,hadm_id,icustay_id,aniongap_min,aniongap_max,albumin_min,albumin_max,bands_min,bands_max,bicarbonate_min,...,inr_min,inr_max,pt_min,pt_max,sodium_min,sodium_max,bun_min,bun_max,wbc_min,wbc_max
2,4,185777,294638,15.0,15.0,2.8,2.8,NaN,NaN,21.0,...,1.1,1.1,12.8,12.8,141.0,141.0,10.0,10.0,9.7,9.7
17,19,109235,273430,15.0,15.0,NaN,NaN,NaN,NaN,22.0,...,1.2,1.3,13.6,14.0,138.0,138.0,25.0,25.0,10.1,10.1
20,21,111970,216859,15.0,19.0,2.1,2.1,1.0,1.0,23.0,...,1.7,1.8,18.1,19.1,137.0,140.0,49.0,50.0,30.5,41.2
25,25,129635,203487,7.0,19.0,3.2,3.2,NaN,NaN,21.0,...,1.0,1.1,12.8,13.0,134.0,139.0,34.0,54.0,11.6,13.0
26,26,197661,244882,12.0,14.0,NaN,NaN,NaN,NaN,24.0,...,1.5,2.4,15.2,18.9,139.0,143.0,31.0,33.0,8.2,8.2
30,31,128652,254478,9.0,10.0,2.7,2.7,NaN,NaN,24.0,...,1.1,1.2,13.0,13.2,128.0,133.0,11.0,13.0,6.9,6.9
32,33,176176,296681,9.0,16.0,3.9,3.9,6.0,12.0,25.0,...,1.2,1.2,13.3,13.3,137.0,140.0,17.0,21.0,26.4,33.5
61,61,176332,252348,11.0,11.0,NaN,NaN,1.0,1.0,23.0,...,0.9,0.9,11.6,11.6,140.0,140.0,17.0,17.0,1.6,1.6
63,62,116009,216609,9.0,13.0,2.7,2.7,7.0,21.0,22.0,...,1.1,1.2,13.0,13.7,136.0,139.0,19.0,22.0,2.0,14.4
65,64,172056,232593,8.0,15.0,2.3,2.3,19.0,19.0,16.0,...,1.1,1.6,12.9,15.6,135.0,140.0,12.0,26.0,12.2,18.1


time: 3min 15s


## Vital Sign

In [88]:
##-- This query pivots the vital signs for the first 24 hours of a patient's stay
##-- Vital signs include heart rate, blood pressure, respiration rate, and temperature

f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/vitalsign.sql', 'r')
vitalsSQL = f.read()
cur.execute('rollback')
cur.execute(vitalsSQL)
vitalsFirstDayTable = sql_exe_show('select * from vitalsfirstday;')
vitalsFirstDayDf= sql_to_df_icu(vitalsFirstDayTable)
vitalsFirstDayDf[0:500]


,subject_id,hadm_id,icustay_id,heartrate_min,heartrate_max,heartrate_mean,sysbp_min,sysbp_max,sysbp_mean,diasbp_min,...,resprate_mean,tempc_min,tempc_max,tempc_mean,spo2_min,spo2_max,spo2_mean,glucose_min,glucose_max,glucose_mean
2,4,185777,294638,74.0,111.0,89.217391,97.0,139.0,118.000000,55.0,...,25.352941,36.333334,37.444445,36.844445,91.0,100.0,97.500000,153.0,266.0,199.666667
15,19,109235,273430,76.0,114.0,98.500000,118.0,163.0,146.350000,44.0,...,20.500000,36.611112,37.277777,36.851852,96.0,100.0,98.150000,139.0,139.0,139.000000
18,21,111970,216859,66.0,83.0,74.971429,78.0,125.0,97.787879,21.0,...,16.885714,36.944444,37.944446,37.411111,79.0,100.0,96.323529,138.0,213.0,171.000000
23,25,129635,203487,49.0,104.0,72.906250,62.0,125.0,100.000000,28.0,...,16.031250,35.722224,36.777776,36.333333,91.0,100.0,97.096774,61.0,364.0,164.766667
24,26,197661,244882,60.0,65.0,62.476190,89.0,111.0,98.047619,24.0,...,19.714286,37.222222,37.722223,37.541666,96.0,98.0,97.090909,118.0,184.0,136.600000
28,31,128652,254478,48.0,54.0,51.230769,85.0,146.0,114.153846,48.0,...,12.066667,36.833335,37.055554,36.958333,97.0,100.0,98.714286,101.0,119.0,108.142857
30,33,176176,296681,58.0,100.0,66.800000,96.0,129.0,113.920000,41.0,...,17.400000,36.611112,37.444445,37.013889,92.0,99.0,95.440000,92.0,115.0,103.500000
57,61,176332,252348,73.0,116.0,94.000000,81.0,116.0,93.565217,40.0,...,20.814815,36.666667,38.888889,38.111112,97.0,100.0,98.555556,95.0,95.0,95.000000
59,62,116009,216609,45.0,86.0,55.840909,87.0,127.0,100.545455,58.0,...,21.909091,35.388887,36.444444,36.034722,93.0,100.0,98.210526,135.0,221.0,178.055556
61,64,172056,232593,88.0,113.0,100.259259,104.0,118.0,110.826087,41.0,...,17.037037,36.444444,39.111112,38.148148,94.0,100.0,97.666667,111.0,188.0,146.000000


time: 8min 58s


## Glasgow Coma Scale

In [ ]:
# -- ITEMIDs used:

# -- CAREVUE
# --    723 as GCSVerbal
# --    454 as GCSMotor
# --    184 as GCSEyes

# -- METAVISION
# --    223900 GCS - Verbal Response
# --    223901 GCS - Motor Response
# --    220739 GCS - Eye Opening

# -- The code combines the ITEMIDs into the carevue itemids, then pivots those
# -- So 223900 is changed to 723, then the ITEMID 723 is pivoted to form GCSVerbal

# -- Note:
# --  The GCS for sedated patients is defaulted to 15 in this code.
# --  This is in line with how the data is meant to be collected.
# --  e.g., from the SAPS II publication:
# --    For sedated patients, the Glasgow Coma Score before sedation was used.
# --    This was ascertained either from interviewing the physician who ordered the sedation,
# --    or by reviewing the patient's medical record.



In [89]:
f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/gcs1.sql', 'r')
gcsSQL = f.read()
cur.execute('rollback')
cur.execute(gcsSQL)
gcsFirstDayTable = sql_exe_show('select * from gcsfirstday;')
gcsFirstDayDf= sql_to_df_icu(gcsFirstDayTable)
gcsFirstDayDf[0:500]


,subject_id,hadm_id,icustay_id,mingcs,gcsmotor,gcsverbal,gcseyes,endotrachflag
7,28448,177527,200012,15.0,6.0,5.0,4.0,0.0
12,21789,112486,200019,7.0,5.0,1.0,1.0,0.0
25,9960,159243,200036,3.0,1.0,1.0,1.0,0.0
39,25225,147080,200055,14.0,6.0,5.0,3.0,0.0
41,28494,199133,200059,15.0,6.0,0.0,3.0,1.0
45,99052,129142,200063,15.0,1.0,0.0,1.0,1.0
46,19891,112840,200065,15.0,1.0,0.0,1.0,1.0
60,61871,124231,200087,15.0,6.0,5.0,4.0,0.0
64,30284,136554,200095,15.0,4.0,0.0,4.0,1.0
67,52619,175374,200099,14.0,6.0,5.0,3.0,0.0


time: 1min 27s


## Renal replacement therapy

In [90]:
f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/rrt.sql', 'r')
rrtSQL = f.read()
cur.execute('rollback')
cur.execute(rrtSQL)
rrtFirstDayTable = sql_exe_show('select * from rrtfirstday;')
rrtFirstDayDf= sql_to_df_icu(rrtFirstDayTable)
rrtFirstDayDf[0:500]


,subject_id,hadm_id,icustay_id,rrt
7,28448,177527,200012,0
12,21789,112486,200019,0
25,9960,159243,200036,0
39,25225,147080,200055,1
41,28494,199133,200059,0
45,99052,129142,200063,0
46,19891,112840,200065,1
60,61871,124231,200087,0
64,30284,136554,200095,0
67,52619,175374,200099,0


time: 20.4 s


In [ ]:
#attempting to write a fxn to perform all this, but the sql_exe_show line requires more customization
# Def file_to_df(filename_no_extension):
#     f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/%s.sql', 'r') %(filename_no_extension)
#     filenameSQL = f.read()
#     cur.execute('rollback')
#     cur.execute(filenameSQL)
#     filenameFirstDayTable = sql_exe_show('select * from vitalsfirstday;')
#     vitalsFirstDayDf= sql_to_df_icu(vitalsFirstDayTable)
#     vitalsFirstDayDf[0:500]

## Urine Output

In [ ]:
#Purpose: Create a view of the urine output for each ICUSTAY_ID over the first 24 hours.

In [91]:
f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/urine.sql', 'r')
urineSQL = f.read()
cur.execute('rollback')
cur.execute(urineSQL)
urineFirstDayTable = sql_exe_show('select * from uofirstday;')
urineFirstDayDf = sql_to_df_icu(urineFirstDayTable)
urineFirstDayDf[0:500]



,subject_id,hadm_id,icustay_id,urineoutput
1,4,185777,294638,2150.0
12,19,109235,273430,940.0
15,21,111970,216859,290.0
20,25,129635,203487,3025.0
21,26,197661,244882,875.0
24,31,128652,254478,3345.0
26,33,176176,296681,2356.0
49,61,176332,252348,1150.0
51,62,116009,216609,718.0
53,64,172056,232593,5695.0


time: 13.2 s


## Blood Gas Test

In [ ]:
# -- The aim of this query is to pivot entries related to blood gases and
# -- chemistry values which were found in LABEVENTS

# -- things to check:
# --  when a mixed venous/arterial blood sample are taken at the same time, is the store time different?


In [92]:
f = open('/Users/geickelb1/Desktop/PhD_Misc/HSIP_442_Yuan_Lao/project/sql/bloodgas.sql', 'r')
bgSQL = f.read()
cur.execute('rollback')
cur.execute(bgSQL)
bgFirstDayTable = sql_exe_show('select * from bloodgasfirstday;')
bgFirstDayDf = sql_to_df_icu(bgFirstDayTable)
bgFirstDayDf[0:500]


NameError: name 'bgFirstDay' is not defined

time: 1min 56s


,subject_id,hadm_id,icustay_id,charttime,specimen,aado2,baseexcess,bicarbonate,totalco2,carboxyhemoglobin,...,peep,ph,po2,potassium,requiredo2,sodium,temperature,tidalvolume,ventilationrate,ventilator
20,4,185777,294638,2191-03-15 21:23:00,ART,NaN,NaN,NaN,21.0,NaN,...,NaN,7.47,74.0,NaN,NaN,NaN,38.9,NaN,None,None
62,19,109235,273430,NaT,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
82,21,111970,216859,2135-01-30 19:18:00,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
101,25,129635,203487,2160-11-02 06:05:00,ART,NaN,NaN,NaN,26.0,NaN,...,NaN,7.39,102.0,NaN,NaN,NaN,NaN,NaN,None,None
102,25,129635,203487,2160-11-02 07:59:00,ART,NaN,NaN,NaN,26.0,NaN,...,NaN,7.33,46.0,NaN,NaN,NaN,NaN,NaN,None,None
103,25,129635,203487,2160-11-02 08:51:00,ART,NaN,NaN,NaN,26.0,NaN,...,NaN,7.34,96.0,NaN,NaN,NaN,37.0,NaN,None,None
104,25,129635,203487,2160-11-02 09:23:00,MIX,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
105,25,129635,203487,2160-11-02 09:25:00,VEN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
106,25,129635,203487,2160-11-02 10:36:00,ART,NaN,NaN,NaN,26.0,NaN,...,NaN,7.28,167.0,NaN,NaN,NaN,NaN,NaN,None,None
107,25,129635,203487,2160-11-02 11:06:00,ART,NaN,NaN,NaN,26.0,NaN,...,NaN,7.29,71.0,NaN,NaN,NaN,NaN,NaN,None,None


time: 58.6 ms


bgFirstDayDf
urineFirstDayDf
rrtFirstDayDf
gcsFirstDayDf
vitalsFirstDayDf
labsFirstDayDf
heightFirstDayDf


# clinical variable QC

In [103]:
lab_bg_overlap_list=list(labsFirstDayDf[labsFirstDayDf['subject_id'].isin(list(
    bgFirstDayDf['subject_id'].unique()))]['subject_id'].unique())

time: 22.1 ms


In [131]:
#bgFirstDayDf
#(labsFirstDayDf[labsFirstDayDf['subject_id'].isin(lab_bg_overlap_list)])
(labsFirstDayDf[labsFirstDayDf['subject_id']==99912])[['potassium_min','potassium_max','icustay_id','subject_id','hadm_id']]

,potassium_min,potassium_max,icustay_id,subject_id,hadm_id
61504,3.9,4.6,236183,99912,189380


time: 10.1 ms


In [130]:
#5-9-18
#(bgFirstDayDf[bgFirstDayDf['subject_id']==(lab_bg_overlap_list)])
#(bgFirstDayDf[bgFirstDayDf['subject_id']==4])[['bicarbonate','specimen','charttime']]
(bgFirstDayDf[bgFirstDayDf['subject_id']==99912])[['potassium','specimen','charttime','icustay_id','subject_id','hadm_id']]
#bgFirstDayDf
#99966 and 99912 subject id has potassium


#qc, checking overlapping categories between bgFirstDayDf and labsFirstDatDf
#subject_id has potassium 4.7 on 2191-08-23 21:38:00 in bgFirstDayDf 
#subject_id has potassium 4.4-4.9 in labsFirstDayDf. 


(ie.intime - interval '6' hour) AND (ie.intime + interval '1' day)


#labs: # -- This query pivots lab values taken in the first 24 hours of a patient's stay
# -- Have already confirmed that the unit of measurement is always the same: null or the correct unit.
#these are from labevents
#labs taken between 6 hours prior to admin and 1 day after admission. 

#bloodgas: # -- The aim of this query is to pivot entries related to blood gases and
# -- chemistry values which were found in LABEVENTS
#labs taken between 6 hours prior to admin and 1 day after admission. 

#ok so labs uses itemid of either 50822 & 50971 where as bloodgas only uses 50822. 
#50822 is postassium, whole blood (category bloodgas) while 50971 is potassium in blood (category chemistry)

,potassium,specimen,charttime,icustay_id,subject_id,hadm_id
243343,4.4,None,2153-04-13 13:37:00,236183,99912,189380
243344,NaN,None,2153-04-13 16:26:00,236183,99912,189380
243345,NaN,None,2153-04-13 16:50:00,236183,99912,189380
243346,NaN,None,2153-04-13 18:36:00,236183,99912,189380
243347,NaN,None,2153-04-13 20:53:00,236183,99912,189380
243348,NaN,None,2153-04-13 21:54:00,236183,99912,189380
243349,NaN,None,2153-04-14 00:20:00,236183,99912,189380
243350,NaN,None,2153-04-14 01:50:00,236183,99912,189380
243351,NaN,None,2153-04-14 04:32:00,236183,99912,189380
243352,NaN,None,2153-04-14 06:38:00,236183,99912,189380


time: 19.2 ms


# test model building

In [133]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
%matplotlib notebook

time: 304 ms


In [ ]:
final_pt_df2

In [135]:
bgFirstDayDf.head()

,subject_id,hadm_id,icustay_id,charttime,specimen,aado2,baseexcess,bicarbonate,totalco2,carboxyhemoglobin,...,peep,ph,po2,potassium,requiredo2,sodium,temperature,tidalvolume,ventilationrate,ventilator
20,4,185777,294638,2191-03-15 21:23:00,ART,NaN,NaN,NaN,21.0,NaN,...,NaN,7.47,74.0,NaN,NaN,NaN,38.9,NaN,None,None
62,19,109235,273430,NaT,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
82,21,111970,216859,2135-01-30 19:18:00,None,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None,None
101,25,129635,203487,2160-11-02 06:05:00,ART,NaN,NaN,NaN,26.0,NaN,...,NaN,7.39,102.0,NaN,NaN,NaN,NaN,NaN,None,None
102,25,129635,203487,2160-11-02 07:59:00,ART,NaN,NaN,NaN,26.0,NaN,...,NaN,7.33,46.0,NaN,NaN,NaN,NaN,NaN,None,None


time: 27.5 ms


In [132]:
test_merge= (
    pd.merge(
        final_pt_df2,
        bgFirstDayDf,
        left_on= 'icustay_id', right_on='icustay_id', how = 'left')
)

time: 527 ms


In [134]:
from sklearn.ensemble import RandomForestClassifier

time: 333 ms


In [ ]:
from sklearn.datasets import load_iris
from sklearn.ensemble import RandomForestClassifier
iris = load_iris()
#rf_clf = RandomForestClassifier().fit(iris.data, iris.target)

iris_df = sns.load_dataset("iris")

iris_df = sns.load_dataset("iris")
g = sns.PairGrid(iris_df, hue='species', 
                 vars=['sepal_length','sepal_width',
                       'petal_length','petal_width'], 
                 palette=sns.xkcd_palette(xkcd_colors), 
                 diag_sharey=False)
g = g.map_upper(plt.scatter, alpha=0.7)
g = g.map_lower(kde_color_plot)
g = g.map_diag(sns.kdeplot, shade=True)

In [138]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score


time: 21.9 ms


In [140]:

#imputing test code,   
import numpy as np

from sklearn.datasets import load_boston
from sklearn.ensemble import RandomForestRegressor
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer
from sklearn.model_selection import cross_val_score

rng = np.random.RandomState(0)

dataset = load_boston()
X_full, y_full = dataset.data, dataset.target
n_samples = X_full.shape[0]
n_features = X_full.shape[1]
# Estimate the score on the entire dataset, with no missing values
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X_full, y_full).mean()
print("Score with the entire dataset = %.2f" % score)

# Add missing values in 75% of the lines
missing_rate = 0.75
n_missing_samples = int(np.floor(n_samples * missing_rate))
missing_samples = np.hstack((np.zeros(n_samples - n_missing_samples,
                                      dtype=np.bool),
                             np.ones(n_missing_samples,
                                     dtype=np.bool)))
rng.shuffle(missing_samples)
missing_features = rng.randint(0, n_features, n_missing_samples)

# Estimate the score without the lines containing missing values
X_filtered = X_full[~missing_samples, :]
y_filtered = y_full[~missing_samples]
estimator = RandomForestRegressor(random_state=0, n_estimators=100)
score = cross_val_score(estimator, X_filtered, y_filtered).mean()
print("Score without the samples containing missing values = %.2f" % score)

# Estimate the score after imputation of the missing values
X_missing = X_full.copy()
X_missing[np.where(missing_samples)[0], missing_features] = 0
y_missing = y_full.copy()
estimator = Pipeline([("imputer", Imputer(missing_values=0,
                                          strategy="mean",
                                          axis=0)),
                      ("forest", RandomForestRegressor(random_state=0,
                                                       n_estimators=100))])
score = cross_val_score(estimator, X_missing, y_missing).mean()
print("Score after imputation of the missing values = %.2f" % score)

Score with the entire dataset = 0.56
Score without the samples containing missing values = 0.48
Score after imputation of the missing values = 0.57
time: 1.79 s


In [156]:
#test_merge.data
#dataset
#X_full
# , y_full = dataset.data, dataset.target
# n_samples = X_full.shape[0]
# n_features = X_full.shape[1]
#dataset
pd.DataFrame(data=dataset)

ValueError: arrays must all be same length

time: 13.2 ms


# altering the sql files to search for all instances within 48 hours

In [194]:
#labs
labs_all48_sql = query_schema + """

SELECT *

FROM
( -- begin query that extracts the data
  SELECT ie.subject_id, ie.hadm_id, ie.icustay_id, le.charttime
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
  , CASE
        WHEN itemid = 50868 THEN 'ANION GAP'
        WHEN itemid = 50862 THEN 'ALBUMIN'
        WHEN itemid = 51144 THEN 'BANDS'
        WHEN itemid = 50882 THEN 'BICARBONATE'
        WHEN itemid = 50885 THEN 'BILIRUBIN'
        WHEN itemid = 50912 THEN 'CREATININE'
        WHEN itemid = 50806 THEN 'CHLORIDE'
        WHEN itemid = 50902 THEN 'CHLORIDE'
        WHEN itemid = 50809 THEN 'GLUCOSE'
        WHEN itemid = 50931 THEN 'GLUCOSE'
        WHEN itemid = 50810 THEN 'HEMATOCRIT'
        WHEN itemid = 51221 THEN 'HEMATOCRIT'
        WHEN itemid = 50811 THEN 'HEMOGLOBIN'
        WHEN itemid = 51222 THEN 'HEMOGLOBIN'
        WHEN itemid = 50813 THEN 'LACTATE'
        WHEN itemid = 51265 THEN 'PLATELET'
        WHEN itemid = 50822 THEN 'POTASSIUM'
        WHEN itemid = 50971 THEN 'POTASSIUM'
        WHEN itemid = 51275 THEN 'PTT'
        WHEN itemid = 51237 THEN 'INR'
        WHEN itemid = 51274 THEN 'PT'
        WHEN itemid = 50824 THEN 'SODIUM'
        WHEN itemid = 50983 THEN 'SODIUM'
        WHEN itemid = 51006 THEN 'BUN'
        WHEN itemid = 51300 THEN 'WBC'
        WHEN itemid = 51301 THEN 'WBC'
      ELSE null
    END AS label
  , -- add in some sanity checks on the values
  -- the where clause below requires all valuenum to be > 0, so these are only upper limit checks
    CASE
      WHEN itemid = 50862 and valuenum >    10 THEN null -- g/dL 'ALBUMIN'
      WHEN itemid = 50868 and valuenum > 10000 THEN null -- mEq/L 'ANION GAP'
      WHEN itemid = 51144 and valuenum <     0 THEN null -- immature band forms, %
      WHEN itemid = 51144 and valuenum >   100 THEN null -- immature band forms, %
      WHEN itemid = 50882 and valuenum > 10000 THEN null -- mEq/L 'BICARBONATE'
      WHEN itemid = 50885 and valuenum >   150 THEN null -- mg/dL 'BILIRUBIN'
      WHEN itemid = 50806 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50902 and valuenum > 10000 THEN null -- mEq/L 'CHLORIDE'
      WHEN itemid = 50912 and valuenum >   150 THEN null -- mg/dL 'CREATININE'
      WHEN itemid = 50809 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50931 and valuenum > 10000 THEN null -- mg/dL 'GLUCOSE'
      WHEN itemid = 50810 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 51221 and valuenum >   100 THEN null -- % 'HEMATOCRIT'
      WHEN itemid = 50811 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 51222 and valuenum >    50 THEN null -- g/dL 'HEMOGLOBIN'
      WHEN itemid = 50813 and valuenum >    50 THEN null -- mmol/L 'LACTATE'
      WHEN itemid = 51265 and valuenum > 10000 THEN null -- K/uL 'PLATELET'
      WHEN itemid = 50822 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 50971 and valuenum >    30 THEN null -- mEq/L 'POTASSIUM'
      WHEN itemid = 51275 and valuenum >   150 THEN null -- sec 'PTT'
      WHEN itemid = 51237 and valuenum >    50 THEN null -- 'INR'
      WHEN itemid = 51274 and valuenum >   150 THEN null -- sec 'PT'
      WHEN itemid = 50824 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 50983 and valuenum >   200 THEN null -- mEq/L == mmol/L 'SODIUM'
      WHEN itemid = 51006 and valuenum >   300 THEN null -- 'BUN'
      WHEN itemid = 51300 and valuenum >  1000 THEN null -- 'WBC'
      WHEN itemid = 51301 and valuenum >  1000 THEN null -- 'WBC'
	  --WHEN ie.subject_id not in (24915,24942,24968,24776,24714,24824) THEN null
    ELSE le.valuenum
    END AS valuenum

  FROM mimiciii.icustays ie
	--where ie.subject_id in (24915,24942,24968,24776,24714,24824)
  LEFT JOIN mimiciii.labevents le
    ON le.subject_id = ie.subject_id AND le.hadm_id = ie.hadm_id
    AND le.charttime BETWEEN (ie.intime - interval '6' hour) AND (ie.intime + interval '2' day)
	AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt. 
    AND le.ITEMID in
    (
      -- comment is: LABEL | CATEGORY | FLUID | NUMBER OF ROWS IN LABEVENTS
      50868, -- ANION GAP | CHEMISTRY | BLOOD | 769895
      50862, -- ALBUMIN | CHEMISTRY | BLOOD | 146697
      51144, -- BANDS - hematology
      50882, -- BICARBONATE | CHEMISTRY | BLOOD | 780733
      50885, -- BILIRUBIN, TOTAL | CHEMISTRY | BLOOD | 238277
      50912, -- CREATININE | CHEMISTRY | BLOOD | 797476
      50902, -- CHLORIDE | CHEMISTRY | BLOOD | 795568
      50806, -- CHLORIDE, WHOLE BLOOD | BLOOD GAS | BLOOD | 48187
      50931, -- GLUCOSE | CHEMISTRY | BLOOD | 748981
      50809, -- GLUCOSE | BLOOD GAS | BLOOD | 196734
      51221, -- HEMATOCRIT | HEMATOLOGY | BLOOD | 881846
      50810, -- HEMATOCRIT, CALCULATED | BLOOD GAS | BLOOD | 89715
      51222, -- HEMOGLOBIN | HEMATOLOGY | BLOOD | 752523
      50811, -- HEMOGLOBIN | BLOOD GAS | BLOOD | 89712
      50813, -- LACTATE | BLOOD GAS | BLOOD | 187124
      51265, -- PLATELET COUNT | HEMATOLOGY | BLOOD | 778444
      50971, -- POTASSIUM | CHEMISTRY | BLOOD | 845825
      50822, -- POTASSIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 192946
      51275, -- PTT | HEMATOLOGY | BLOOD | 474937
      51237, -- INR(PT) | HEMATOLOGY | BLOOD | 471183
      51274, -- PT | HEMATOLOGY | BLOOD | 469090
      50983, -- SODIUM | CHEMISTRY | BLOOD | 808489
      50824, -- SODIUM, WHOLE BLOOD | BLOOD GAS | BLOOD | 71503
      51006, -- UREA NITROGEN | CHEMISTRY | BLOOD | 791925
      51301, -- WHITE BLOOD CELLS | HEMATOLOGY | BLOOD | 753301
      51300  -- WBC COUNT | HEMATOLOGY | BLOOD | 2371
    )
    AND valuenum IS NOT null AND valuenum > 0 -- lab values cannot be 0 and cannot be negative
) pvt;

    """.format(tuple(patients))
    
labs_all48_df=pd.read_sql_query(labs_all48_sql,conn)    


#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)
#will need to do this for all of my sql queres.


###guessing this isn't correct yet. need to adjust by removing the group_by's. addin charttime, and remove the max/min categories.

time: 41.4 s


In [195]:
labs_all48_df

,subject_id,hadm_id,icustay_id,charttime,label,valuenum
0,4,185777,294638,2191-03-15 21:23:00,HEMATOCRIT,32.0
1,4,185777,294638,2191-03-15 21:23:00,HEMOGLOBIN,10.5
2,4,185777,294638,2191-03-15 21:23:00,LACTATE,2.1
3,4,185777,294638,2191-03-16 05:42:00,ALBUMIN,2.8
4,4,185777,294638,2191-03-16 05:42:00,ANION GAP,15.0
5,4,185777,294638,2191-03-16 05:42:00,BICARBONATE,21.0
6,4,185777,294638,2191-03-16 05:42:00,BILIRUBIN,1.9
7,4,185777,294638,2191-03-16 05:42:00,CHLORIDE,108.0
8,4,185777,294638,2191-03-16 05:42:00,CREATININE,0.5
9,4,185777,294638,2191-03-16 05:42:00,GLUCOSE,183.0


time: 32.3 ms


In [179]:
#blood gas, updated

bg_all48_sql = query_schema + """
select pvt.SUBJECT_ID, pvt.HADM_ID, pvt.ICUSTAY_ID, pvt.CHARTTIME, label, valuenum

from
( -- begin query that extracts the data
  select ie.subject_id, ie.hadm_id, ie.icustay_id
  -- here we assign labels to ITEMIDs
  -- this also fuses together multiple ITEMIDs containing the same data
      , case
        when itemid = 50800 then 'SPECIMEN'
        when itemid = 50801 then 'AADO2'
        when itemid = 50802 then 'BASEEXCESS'
        when itemid = 50803 then 'BICARBONATE'
        when itemid = 50804 then 'TOTALCO2'
        when itemid = 50805 then 'CARBOXYHEMOGLOBIN'
        when itemid = 50806 then 'CHLORIDE'
        when itemid = 50808 then 'CALCIUM'
        when itemid = 50809 then 'GLUCOSE'
        when itemid = 50810 then 'HEMATOCRIT'
        when itemid = 50811 then 'HEMOGLOBIN'
        when itemid = 50812 then 'INTUBATED'
        when itemid = 50813 then 'LACTATE'
        when itemid = 50814 then 'METHEMOGLOBIN'
        when itemid = 50815 then 'O2FLOW'
        when itemid = 50816 then 'FIO2'
        when itemid = 50817 then 'SO2' -- OXYGENSATURATION
        when itemid = 50818 then 'PCO2'
        when itemid = 50819 then 'PEEP'
        when itemid = 50820 then 'PH'
        when itemid = 50821 then 'PO2'
        when itemid = 50822 then 'POTASSIUM'
        when itemid = 50823 then 'REQUIREDO2'
        when itemid = 50824 then 'SODIUM'
        when itemid = 50825 then 'TEMPERATURE'
        when itemid = 50826 then 'TIDALVOLUME'
        when itemid = 50827 then 'VENTILATIONRATE'
        when itemid = 50828 then 'VENTILATOR'
        else null
        end as label
        , charttime
        , value
        -- add in some sanity checks on the values
        , case
          when valuenum <= 0 then null
          when itemid = 50810 and valuenum > 100 then null -- hematocrit
          when itemid = 50816 and valuenum > 100 then null -- FiO2
          when itemid = 50817 and valuenum > 100 then null -- O2 sat
          when itemid = 50815 and valuenum >  70 then null -- O2 flow
          when itemid = 50821 and valuenum > 800 then null -- PO2
           -- conservative upper limit
        else valuenum
        end as valuenum

    from mimiciii.icustays ie
    left join mimiciii.labevents le
      on le.subject_id = ie.subject_id and le.hadm_id = ie.hadm_id
      and le.charttime between (ie.intime - interval '6' hour) and (ie.intime + interval '2' day)
  	  and ie.subject_id in {} --!!THIS IS THE code to get for only specific pt. 
      and le.ITEMID in
      -- blood gases
      (
        50800, 50801, 50802, 50803, 50804, 50805, 50806, 50807, 50808, 50809
        , 50810, 50811, 50812, 50813, 50814, 50815, 50816, 50817, 50818, 50819
        , 50820, 50821, 50822, 50823, 50824, 50825, 50826, 50827, 50828
        , 51545
      )
) pvt
group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.CHARTTIME, label, valuenum
order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.CHARTTIME, label, valuenum
; """.format(tuple(patients))
    
bg_all48_df=pd.read_sql_query(bg_all48_sql,conn)    
bg_all48_df= bg_all48_df[bg_all48_df['label'].notnull()]# 44451 null rows removed. 
#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)

time: 3min 34s


In [163]:
# #blood gas,  mid-modification, saved for reference

# bg_all48_sql = query_schema + """
# -- The aim of this query is to pivot entries related to blood gases and
# -- chemistry values which were found in LABEVENTS

# -- things to check:
# --  when a mixed venous/arterial blood sample are taken at the same time, is the store time different?

# --DROP MATERIALIZED VIEW IF EXISTS bloodgasfirstday CASCADE;
# --create materialized view bloodgasfirstday as
# select pvt.SUBJECT_ID, pvt.HADM_ID, pvt.ICUSTAY_ID, pvt.CHARTTIME

# , max(case when label = 'SPECIMEN' then value else null end) as SPECIMEN
# , max(case when label = 'AADO2' then valuenum else null end) as AADO2
# , max(case when label = 'BASEEXCESS' then valuenum else null end) as BASEEXCESS
# , max(case when label = 'BICARBONATE' then valuenum else null end) as BICARBONATE
# , max(case when label = 'TOTALCO2' then valuenum else null end) as TOTALCO2
# , max(case when label = 'CARBOXYHEMOGLOBIN' then valuenum else null end) as CARBOXYHEMOGLOBIN
# , max(case when label = 'CHLORIDE' then valuenum else null end) as CHLORIDE
# , max(case when label = 'CALCIUM' then valuenum else null end) as CALCIUM
# , max(case when label = 'GLUCOSE' then valuenum else null end) as GLUCOSE
# , max(case when label = 'HEMATOCRIT' then valuenum else null end) as HEMATOCRIT
# , max(case when label = 'HEMOGLOBIN' then valuenum else null end) as HEMOGLOBIN
# , max(case when label = 'INTUBATED' then valuenum else null end) as INTUBATED
# , max(case when label = 'LACTATE' then valuenum else null end) as LACTATE
# , max(case when label = 'METHEMOGLOBIN' then valuenum else null end) as METHEMOGLOBIN
# , max(case when label = 'O2FLOW' then valuenum else null end) as O2FLOW
# , max(case when label = 'FIO2' then valuenum else null end) as FIO2
# , max(case when label = 'SO2' then valuenum else null end) as SO2 -- OXYGENSATURATION
# , max(case when label = 'PCO2' then valuenum else null end) as PCO2
# , max(case when label = 'PEEP' then valuenum else null end) as PEEP
# , max(case when label = 'PH' then valuenum else null end) as PH
# , max(case when label = 'PO2' then valuenum else null end) as PO2
# , max(case when label = 'POTASSIUM' then valuenum else null end) as POTASSIUM
# , max(case when label = 'REQUIREDO2' then valuenum else null end) as REQUIREDO2
# , max(case when label = 'SODIUM' then valuenum else null end) as SODIUM
# , max(case when label = 'TEMPERATURE' then valuenum else null end) as TEMPERATURE
# , max(case when label = 'TIDALVOLUME' then valuenum else null end) as TIDALVOLUME
# , max(case when label = 'VENTILATIONRATE' then valuenum else null end) as VENTILATIONRATE
# , max(case when label = 'VENTILATOR' then valuenum else null end) as VENTILATOR
# from
# ( -- begin query that extracts the data
#   select ie.subject_id, ie.hadm_id, ie.icustay_id
#   -- here we assign labels to ITEMIDs
#   -- this also fuses together multiple ITEMIDs containing the same data
#       , case
#         when itemid = 50800 then 'SPECIMEN'
#         when itemid = 50801 then 'AADO2'
#         when itemid = 50802 then 'BASEEXCESS'
#         when itemid = 50803 then 'BICARBONATE'
#         when itemid = 50804 then 'TOTALCO2'
#         when itemid = 50805 then 'CARBOXYHEMOGLOBIN'
#         when itemid = 50806 then 'CHLORIDE'
#         when itemid = 50808 then 'CALCIUM'
#         when itemid = 50809 then 'GLUCOSE'
#         when itemid = 50810 then 'HEMATOCRIT'
#         when itemid = 50811 then 'HEMOGLOBIN'
#         when itemid = 50812 then 'INTUBATED'
#         when itemid = 50813 then 'LACTATE'
#         when itemid = 50814 then 'METHEMOGLOBIN'
#         when itemid = 50815 then 'O2FLOW'
#         when itemid = 50816 then 'FIO2'
#         when itemid = 50817 then 'SO2' -- OXYGENSATURATION
#         when itemid = 50818 then 'PCO2'
#         when itemid = 50819 then 'PEEP'
#         when itemid = 50820 then 'PH'
#         when itemid = 50821 then 'PO2'
#         when itemid = 50822 then 'POTASSIUM'
#         when itemid = 50823 then 'REQUIREDO2'
#         when itemid = 50824 then 'SODIUM'
#         when itemid = 50825 then 'TEMPERATURE'
#         when itemid = 50826 then 'TIDALVOLUME'
#         when itemid = 50827 then 'VENTILATIONRATE'
#         when itemid = 50828 then 'VENTILATOR'
#         else null
#         end as label
#         , charttime
#         , value
#         -- add in some sanity checks on the values
#         , case
#           when valuenum <= 0 then null
#           when itemid = 50810 and valuenum > 100 then null -- hematocrit
#           when itemid = 50816 and valuenum > 100 then null -- FiO2
#           when itemid = 50817 and valuenum > 100 then null -- O2 sat
#           when itemid = 50815 and valuenum >  70 then null -- O2 flow
#           when itemid = 50821 and valuenum > 800 then null -- PO2
#            -- conservative upper limit
#         else valuenum
#         end as valuenum

#     from mimiciii.icustays ie
#     left join mimiciii.labevents le
#       on le.subject_id = ie.subject_id and le.hadm_id = ie.hadm_id
#       and le.charttime between (ie.intime - interval '6' hour) and (ie.intime + interval '2' day)
#       and le.subject_id in  {} --!!THIS IS THE code to get for only specific pt. 
#       and le.ITEMID in
#       -- blood gases
#       (
#         50800, 50801, 50802, 50803, 50804, 50805, 50806, 50807, 50808, 50809
#         , 50810, 50811, 50812, 50813, 50814, 50815, 50816, 50817, 50818, 50819
#         , 50820, 50821, 50822, 50823, 50824, 50825, 50826, 50827, 50828
#         , 51545
#       )
# ) pvt
# group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.CHARTTIME
# order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id, pvt.CHARTTIME; """.format(tuple(patients))
    
# bg_all48_df=pd.read_sql_query(bg_all48_sql,conn)    

# #changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# # into the join at the end. removed the create table at beginning and comitt at the end. 
# #also select * from the initial generated table
# #also also changed the date range to 48 hours after, rather than 24. (2 day)


# ###guessing this isn't correct yet. need to adjust by removing the group_by's. addin charttime, and remove the max/min categories.

time: 50.9 s


In [175]:
#vital

vital_all48_sql = query_schema + """

SELECT *

FROM  (
  select ie.subject_id, ie.hadm_id, ie.icustay_id, ce.charttime --added ce.charttime
  , case
    when itemid in (211,220045) and valuenum > 0 and valuenum < 300 then 'HeartRate' -- HeartRate
    when itemid in (51,442,455,6701,220179,220050) and valuenum > 0 and valuenum < 400 then 'SysBP' -- SysBP
    when itemid in (8368,8440,8441,8555,220180,220051) and valuenum > 0 and valuenum < 300 then 'DiasBP' -- DiasBP
    when itemid in (456,52,6702,443,220052,220181,225312) and valuenum > 0 and valuenum < 300 then 'MeanBP' -- MeanBP
    when itemid in (615,618,220210,224690) and valuenum > 0 and valuenum < 70 then 'RespRate' -- RespRate
    when itemid in (223761,678) and valuenum > 70 and valuenum < 120  then 'TempF' -- TempF, converted to degC in valuenum call
    when itemid in (223762,676) and valuenum > 10 and valuenum < 50  then 'TempC' -- TempC
    when itemid in (646,220277) and valuenum > 0 and valuenum <= 100 then 'SpO2' -- SpO2
    when itemid in (807,811,1529,3745,3744,225664,220621,226537) and valuenum > 0 then 'Glucose' -- Glucose

    else null end as VitalID
      -- convert F to C
  , case when itemid in (223761,678) then (valuenum-32)/1.8 else valuenum end as valuenum

  from mimiciii.icustays ie
  left join mimiciii.chartevents ce
  on ie.subject_id = ce.subject_id and ie.hadm_id = ce.hadm_id and ie.icustay_id = ce.icustay_id
  and ce.charttime between ie.intime and ie.intime + interval '2' day
  and ie.subject_id in  {} --!!THIS IS THE code to get for only specific pt. 
  -- exclude rows marked as error
  and ce.error IS DISTINCT FROM 1
  where ce.itemid in
  (
  -- HEART RATE
  211, --"Heart Rate"
  220045, --"Heart Rate"

  -- Systolic/diastolic

  51, --    Arterial BP [Systolic]
  442, --   Manual BP [Systolic]
  455, --   NBP [Systolic]
  6701, --  Arterial BP #2 [Systolic]
  220179, --    Non Invasive Blood Pressure systolic
  220050, --    Arterial Blood Pressure systolic

  8368, --  Arterial BP [Diastolic]
  8440, --  Manual BP [Diastolic]
  8441, --  NBP [Diastolic]
  8555, --  Arterial BP #2 [Diastolic]
  220180, --    Non Invasive Blood Pressure diastolic
  220051, --    Arterial Blood Pressure diastolic


  -- MEAN ARTERIAL PRESSURE
  456, --"NBP Mean"
  52, --"Arterial BP Mean"
  6702, --  Arterial BP Mean #2
  443, --   Manual BP Mean(calc)
  220052, --"Arterial Blood Pressure mean"
  220181, --"Non Invasive Blood Pressure mean"
  225312, --"ART BP mean"

  -- RESPIRATORY RATE
  618,--    Respiratory Rate
  615,--    Resp Rate (Total)
  220210,-- Respiratory Rate
  224690, --    Respiratory Rate (Total)


  -- SPO2, peripheral
  646, 220277,

  -- GLUCOSE, both lab and fingerstick
  807,--    Fingerstick Glucose
  811,--    Glucose (70-105)
  1529,--   Glucose
  3745,--   BloodGlucose
  3744,--   Blood Glucose
  225664,-- Glucose finger stick
  220621,-- Glucose (serum)
  226537,-- Glucose (whole blood)

  -- TEMPERATURE
  223762, -- "Temperature Celsius"
  676,  -- "Temperature C"
  223761, -- "Temperature Fahrenheit"
  678 --    "Temperature F"

  )
) pvt;
--group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id
--order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id;""".format(tuple(patients))
    
vital_all48_df=pd.read_sql_query(vital_all48_sql,conn)    

#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)



time: 4min 39s


In [176]:
len(vital_all48_df)

6741975

time: 32.7 ms


In [197]:
vital_all48_df.head()

,subject_id,hadm_id,icustay_id,charttime,vitalid,valuenum
0,21,111970,216859,2135-02-01 15:30:00,MeanBP,61.0
1,25,129635,203487,2160-11-02 15:00:00,MeanBP,82.0
2,25,129635,203487,2160-11-02 16:00:00,MeanBP,92.0
3,25,129635,203487,2160-11-02 17:00:00,MeanBP,71.0
4,25,129635,203487,2160-11-03 08:00:00,MeanBP,74.0


time: 168 ms


In [ ]:
#height
##not needed according to nelson


height_all48_sql = query_schema + """
""".format(tuple(patients))
    
height_all48_df=pd.read_sql_query(height_all48_sql,conn)    

#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)



In [ ]:
#weight
#this doesn't need to be changed since we only need weight_admit according to nelson

weight_all48_sql = query_schema + """

""".format(tuple(patients))
    
weight_all48_df=pd.read_sql_query(weight_all48_sql,conn)    

#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)



In [ ]:
#urine
##not needed according to nelson

urine_all48_sql = query_schema + """

""".format(tuple(patients))
    
urine_all48_df=pd.read_sql_query(urine_all48_sql,conn)    

#changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# into the join at the end. removed the create table at beginning and comitt at the end. 
#also select * from the initial generated table
#also also changed the date range to 48 hours after, rather than 24. (2 day)



In [ ]:
#rrt
##not needed according to nelson

In [173]:
# #vital, mid-adjustment. saved for reference. 

# vital_all48_sql = query_schema + """
# SELECT *

# FROM  (
#   select ie.subject_id, ie.hadm_id, ie.icustay_id
#   , case
#     when itemid in (211,220045) and valuenum > 0 and valuenum < 300 then 1 -- HeartRate
#     when itemid in (51,442,455,6701,220179,220050) and valuenum > 0 and valuenum < 400 then 2 -- SysBP
#     when itemid in (8368,8440,8441,8555,220180,220051) and valuenum > 0 and valuenum < 300 then 3 -- DiasBP
#     when itemid in (456,52,6702,443,220052,220181,225312) and valuenum > 0 and valuenum < 300 then 4 -- MeanBP
#     when itemid in (615,618,220210,224690) and valuenum > 0 and valuenum < 70 then 5 -- RespRate
#     when itemid in (223761,678) and valuenum > 70 and valuenum < 120  then 6 -- TempF, converted to degC in valuenum call
#     when itemid in (223762,676) and valuenum > 10 and valuenum < 50  then 6 -- TempC
#     when itemid in (646,220277) and valuenum > 0 and valuenum <= 100 then 7 -- SpO2
#     when itemid in (807,811,1529,3745,3744,225664,220621,226537) and valuenum > 0 then 8 -- Glucose

#     else null end as VitalID
#       -- convert F to C
#   , case when itemid in (223761,678) then (valuenum-32)/1.8 else valuenum end as valuenum

#   from mimiciii.icustays ie
#   left join mimiciii.chartevents ce
#   on ie.subject_id = ce.subject_id and ie.hadm_id = ce.hadm_id and ie.icustay_id = ce.icustay_id
#   and ce.charttime between ie.intime and ie.intime + interval '2' day
#   and ie.subject_id in  {} --!!THIS IS THE code to get for only specific pt. 
#   -- exclude rows marked as error
#   and ce.error IS DISTINCT FROM 1
#   where ce.itemid in
#   (
#   -- HEART RATE
#   211, --"Heart Rate"
#   220045, --"Heart Rate"

#   -- Systolic/diastolic

#   51, --    Arterial BP [Systolic]
#   442, --   Manual BP [Systolic]
#   455, --   NBP [Systolic]
#   6701, --  Arterial BP #2 [Systolic]
#   220179, --    Non Invasive Blood Pressure systolic
#   220050, --    Arterial Blood Pressure systolic

#   8368, --  Arterial BP [Diastolic]
#   8440, --  Manual BP [Diastolic]
#   8441, --  NBP [Diastolic]
#   8555, --  Arterial BP #2 [Diastolic]
#   220180, --    Non Invasive Blood Pressure diastolic
#   220051, --    Arterial Blood Pressure diastolic


#   -- MEAN ARTERIAL PRESSURE
#   456, --"NBP Mean"
#   52, --"Arterial BP Mean"
#   6702, --  Arterial BP Mean #2
#   443, --   Manual BP Mean(calc)
#   220052, --"Arterial Blood Pressure mean"
#   220181, --"Non Invasive Blood Pressure mean"
#   225312, --"ART BP mean"

#   -- RESPIRATORY RATE
#   618,--    Respiratory Rate
#   615,--    Resp Rate (Total)
#   220210,-- Respiratory Rate
#   224690, --    Respiratory Rate (Total)


#   -- SPO2, peripheral
#   646, 220277,

#   -- GLUCOSE, both lab and fingerstick
#   807,--    Fingerstick Glucose
#   811,--    Glucose (70-105)
#   1529,--   Glucose
#   3745,--   BloodGlucose
#   3744,--   Blood Glucose
#   225664,-- Glucose finger stick
#   220621,-- Glucose (serum)
#   226537,-- Glucose (whole blood)

#   -- TEMPERATURE
#   223762, -- "Temperature Celsius"
#   676,  -- "Temperature C"
#   223761, -- "Temperature Fahrenheit"
#   678 --    "Temperature F"

#   )
# ) pvt,
# --group by pvt.subject_id, pvt.hadm_id, pvt.icustay_id
# --order by pvt.subject_id, pvt.hadm_id, pvt.icustay_id;
# """.format(tuple(patients))
    
# vital_all48_df=pd.read_sql_query(vital_all48_sql,conn)    

# #changes to origional, added in the AND le.subject_id in  {} --!!THIS IS THE code to get for only specific pt
# # into the join at the end. removed the create table at beginning and comitt at the end. 
# #also select * from the initial generated table
# #also also changed the date range to 48 hours after, rather than 24. (2 day)





# ######this isn't quite there, need to adjust to grab all values and get time identifier for them. 

time: 5min 31s


In [196]:
pd.DataFrame(vital_all48_df).to_csv('clinical_csv/vital_all48_df.csv')
pd.DataFrame(labs_all48_df).to_csv('clinical_csv/labs_all48_df.csv')
pd.DataFrame(bg_all48_df).to_csv('clinical_csv/bg_all48_df.csv')


time: 1min 3s


# 5/11/18 todo:
1) figure out how to extract the additional features nelson want's

2) determine how to process each datapoint for single value for each feature in model

3) assemble df for scaling, imputing, model building, etc...
